# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [3]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='meditation'
)

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [6]:
import haystack
dir(haystack)

['Answer',
 'BaseComponent',
 'Document',
 'EvaluationResult',
 'Label',
 'ModuleType',
 'MultiLabel',
 'Pipeline',
 'Span',
 'Union',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'document_stores',
 'environment',
 'errors',
 'logging',
 'metadata',
 'modeling',
 'nodes',
 'pd',
 'pipelines',
 'schema',
 'set_pytorch_secure_model_loading',
 'telemetry',
 'utils']

In [4]:
from haystack.nodes import ElasticsearchRetriever
from haystack.nodes import FARMReader

retriever = ElasticsearchRetriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

And now we initialize our ODQA pipeline.

In [5]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [6]:
qa.run(query='What did your grandfather teach?')

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.09s/ Batches]


{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.1212544441223145,
 'answers': [<Answer {'answer': 'good morals and the government of my temper', 'type': 'extractive', 'score': 0.1665751039981842, 'context': 'From my grandfather Verus I learned good morals and the government of my temper.', 'offsets_in_document': [{'start': 36, 'end': 79}], 'offsets_in_context': [{'start': 36, 'end': 79}], 'document_ids': ['ecc466cb76ed8dedab8752cc84e411b4'], 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'thy life under thy mother', 'type': 'extractive', 'score': 0.024128641933202744, 'context': 'Termination of activity, cessation from movement and opinion, and in a sense their death, is no evil. Turn thy thoughts now to the consideration of thy life, thy life as a child, as a youth, thy manhood, thy old age, for in these also every change was a death. Is this anything to fear? Turn thy thoughts now to thy life under thy grandfather, then to thy life under thy mother, then to

In [1]:
qa.run(query='What is the Universe?',
       params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 3}})

NameError: name 'qa' is not defined

In [14]:
qa.run(query='What is art?',
       params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.67s/ Batches]


{'query': 'What is art?',
 'no_ans_gap': 4.066236734390259,
 'answers': [<Answer {'answer': 'To be good', 'type': 'extractive', 'score': 0.7172240018844604, 'context': 'What is thy art? To be good. And how is this accomplished well except by general principles, some about the nature of the universe, and others about the proper constitution of man?', 'offsets_in_document': [{'start': 17, 'end': 27}], 'offsets_in_context': [{'start': 17, 'end': 27}], 'document_ids': ['d82018bb6d9e387d13e62efe36f458c6'], 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'the acts of life', 'type': 'extractive', 'score': 0.5413747429847717, 'context': 'Let it make no difference to thee whether thou art cold or warm, if thou art doing thy duty; and whether thou art drowsy or satisfied with sleep; and whether ill-spoken of or praised; and whether dying or doing something else. For it is one of the acts of life, this act by which we die: it is sufficient then in this act also to do well what we have i